In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pyeCAP
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
import traceback
import os
import dask.array as da
from scipy import signal
from scipy.ndimage import uniform_filter1d
import plotly.express as px

In [ ]:
"Import meta data and TDT files"
"""Get HF meta data from excel file"""
#excel_path = r'E:\HF_Block\20240904_HFBlock_Pig05\20240904_HFBlock_05_Parameters_modified.xlsx'
excel_path = r'G:\Data\HF_Block\HF_Block_Project_summary.xlsx'
metaDF = pd.read_excel(excel_path, sheet_name = 'HF05_metaDF')

"""Remove rows that are not needed for current analysis and reset index of metaDF"""
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'HF Only'].index, inplace = True)  #Removed HF only tanks as they did not have any stim parameters
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Control'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'X'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Thresholding'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Contact ID'].index, inplace = True)
metaDF.reset_index(inplace = True)

"TDT Data Import - List"
#tdt_path = r'E:\HF_Block\20240904_HFBlock_Pig05\HF_Block_Template-240904-083702' #Lab Path
tdt_path = r'G:\Data\HF_Block\20240904_HFBlock_Pig05\HF_Block_Template-240904-083702' #Local Path 
tdt_file_list_all = os.listdir(tdt_path)
tdt_file_list_all.remove('desktop.ini')

#ADI_path = r'E:\HF_Block\20240904_HFBlock_Pig05' #Lab Path
ADI_path = r'G:\Data\HF_Block\20240904_HFBlock_Pig05' #Local path
ADI_file = r'\20240904_HFBlock_Pig05_stats_modified.mat'

raw_phys = pyeCAP.Phys(ADI_path + ADI_file)#, mult_data=False)
ws_stripped_names = [i.strip() for i in raw_phys.ch_names] #This line just removes the white spaces around the names imported
raw_phys = raw_phys.set_ch_names(ws_stripped_names)

"""Ephys data instantiation"""
tdt_path_list = [tdt_path + '\\' + fileNAME for fileNAME in metaDF['TDT Tank']]

#Data Streams
#raw_ECAP = pyeCAP.Ephys(tdt_path_list, stores = 'ECAP')
#raw_ECAP = raw_ECAP.remove_ch(['ECAP 6', 'ECAP 7', 'ECAP 8'])

"""Changes the stim multi-index to utilize the Tank names from tdt_file_list instead [0,1,2,3...]"""
stim = pyeCAP.Stim(tdt_path_list)
stimDF = stim.parameters
#tank_to_params_dict = { tank:params for (tank,params) in zip(metaDF['TDT Tank'].to_list(), stim.parameters.groupby(level=0).groups.values() ) }
#params_to_tank_dict = { param:tank for (param,tank) in zip(stim.parameters.groupby(level=0).groups.keys(), metaDF['TDT Tank'].to_list()) }

"""ECAP/EMG/TriCAP object instantiation"""
#ECAPr = pyeCAP.ECAP(raw_ECAP, stim)
PHYS = pyeCAP.ECAP(raw_phys,stim)
raw_ECG_array = np.squeeze(raw_phys.array[raw_phys._ch_to_index('EKG Raw')].compute() )

In [ ]:
"""Band-pass Filter"""
ECG_fs = 1000
nyq_rate = ECG_fs / 2

cutoffs = [5,15]
sos = signal.butter(4, cutoffs, btype='bandpass', fs = ECG_fs,output='sos')

"""Code which replaces any NaN values by simply replacing them with the first value after the NaN's occur"""
nan_idx = np.argwhere(np.isnan(raw_ECG_array))
print(nan_idx)
#np.nan_to_num(raw_ECG_array, copy=False, nan=raw_ECG_array[nan_idx[-1] + 1])

ECG_BP_filt = signal.sosfiltfilt(sos, raw_ECG_array)
ECG_dvt_filt = np.square(np.diff(ECG_BP_filt,append=0))
#ECG_sq = np.square(ECG_dvt_filt)
#ECG_moving_average = uniform_filter1d(ECG_sq,size=150)

In [ ]:
filt_data, data_diff_sq,data_moving_average = raw_phys.pan_tompkins_algorithm(ecg_ch_name = 'EKG Raw')
filt_data_array = np.squeeze(da.concatenate(filt_data,axis=1).compute())
diff_sq_array = np.squeeze(da.concatenate(data_diff_sq,axis=1).compute())
mwa_array = np.squeeze(da.concatenate(data_moving_average,axis=1).compute())

In [ ]:
fig = plt.figure(num=1, clear=True)
ax = fig.add_subplot()
dask_dv_diff = np.subtract(diff_sq_array,ECG_dvt_filt)
#dask_filt_diff = np.subtract(filt_data_array, ECG_BP_filt)
#ax.plot(raw_phys.time().compute(),dask_dv_diff)
#ax.plot(raw_phys.time().compute(),dask_filt_diff)
ax.plot(raw_phys.time()[5000000:],mwa_array[5000000:], color = 'r')
ax2 = ax.twinx()
ax2.plot(raw_phys.time()[5000000:],dask_dv_diff[5000000:])

In [ ]:
idx = np.nonzero(mwa_array > 1*1e-10)[0]

In [ ]:
idx

In [ ]:
dask_dv_diff = np.subtract(diff_sq_array,ECG_dvt_filt)
fig,ax = plt.subplots(nrows = 4, ncols=1,num=1, clear=True, figsize=(10,10))

#center_idx = idx[-1]
center_idx = 9790000
start_idx = center_idx - (5 * 1000)
stop_idx = center_idx + (5 * 1000)
if start_idx < 0:
    start_idx = 0
print(start_idx, stop_idx)

ax = ax.ravel()
#ax2 = ax.twinx()
#ax2.plot(raw_phys.time()[start_idx:stop_idx],dask_dv_diff[start_idx:stop_idx])
#ax2.plot(raw_phys.time()[start_idx:stop_idx],diff_sq_array[start_idx:stop_idx])
ax[0].plot(raw_phys.time()[start_idx:stop_idx],raw_ECG_array[start_idx:stop_idx])
ax[1].plot(raw_phys.time()[start_idx:stop_idx],filt_data_array[start_idx:stop_idx])
#ax[2].plot(raw_phys.time()[start_idx:stop_idx],diff_sq_array[start_idx:stop_idx])
ax[2].plot(raw_phys.time()[start_idx:stop_idx],dask_dv_diff[start_idx:stop_idx])
ax[3].plot(raw_phys.time()[start_idx:stop_idx],mwa_array[start_idx:stop_idx], color = 'r')
mwa_std = np.std(mwa_array[5000000:])
ax[3].axhline(mwa_std, color='g')
plt.show()

In [ ]:
mwa_std = np.mean(mwa_array)
print(mwa_std)

In [ ]:
mwa_std = np.std(mwa_array[5000000:])
print(mwa_std)

In [ ]:
stop_idx

In [ ]:
ignore = raw_phys.remove_cautery(ecg_ch_name = 'EKG Raw')

In [ ]:
asdf = []
for idx, val in ignore[0]:
    if val > 1:
        asdf.append[idx]

In [ ]:
ignore.A

In [ ]:
x.compute()

In [ ]:
for x in ignore:
    print(x.shape)

In [ ]:
ignore

In [ ]:
asdf = raw_phys.pan_tompkins_algorithm(ecg_ch_name = 'EKG Raw')

In [ ]:
asdf.units

In [ ]:
type(old_ch_names)

In [ ]:
ch_names = [raw_phys.ch_names, 'ECG BP Filtered', 'ECG DiffSq', 'ECG Moving Average']

In [ ]:
old_ch_names.concatenate(['ECG BP Filtered', 'ECG DiffSq', 'ECG Moving Average')

In [ ]:
old_ch_names

In [ ]:
data = [da.concatenate([d1,d2,d3,d4],axis=0) for d1,d2,d3,d4 in zip(og_data,filt_data, data_diff_sq, data_moving_average)]